<a href="https://colab.research.google.com/github/cedamusk/Astrophysics/blob/main/PINNs_Implementation_for_Planetary_motion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch numpy matplotlib seaborn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import seaborn as sns

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class ImprovedPINN(nn.Module):
    def __init__(self):
        super().__init__()
        # Wider network with better initialization
        self.network = nn.Sequential(
            nn.Linear(1, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 2)
        )

        # Xavier initialization
        for layer in self.network:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_normal_(layer.weight)
                nn.init.zeros_(layer.bias)

    def forward(self, t):
        return self.network(t)

    def compute_derivatives(self, t):
        t.requires_grad_(True)
        xy = self.forward(t)

        dxy_dt = torch.autograd.grad(
            xy, t,
            grad_outputs=torch.ones_like(xy),
            create_graph=True
        )[0]

        d2xy_dt2 = torch.autograd.grad(
            dxy_dt, t,
            grad_outputs=torch.ones_like(dxy_dt),
            create_graph=True
        )[0]

        return xy, dxy_dt, d2xy_dt2

def generate_orbital_data(n_points=1000, noise_level=0.005):  # Reduced noise
    """Generate cleaner orbital data"""
    t = np.linspace(0, 10, n_points)

    # Orbital parameters (more stable orbit)
    r = 1.0
    omega = 2*np.pi/5

    # True solution
    x = r * np.cos(omega * t)
    y = r * np.sin(omega * t)

    # Add reduced noise
    x += noise_level * np.random.randn(n_points)
    y += noise_level * np.random.randn(n_points)

    return t, x, y

def physics_loss(model, t, normalize=True):
    """Improved physics loss with normalization"""
    xy, dxy_dt, d2xy_dt2 = model.compute_derivatives(t)

    # Gravitational parameter
    k = 4 * np.pi**2

    # Position vectors
    r = torch.sqrt(xy[:, 0]**2 + xy[:, 1]**2)

    # Physics residuals
    residual_x = d2xy_dt2[:, 0] + k * xy[:, 0] / (r**3)
    residual_y = d2xy_dt2[:, 1] + k * xy[:, 1] / (r**3)

    if normalize:
        # Normalize residuals by the magnitude of terms
        scale = torch.mean(torch.abs(k * xy / (r**3).unsqueeze(1)))
        residual_x = residual_x / scale
        residual_y = residual_y / scale

    return torch.mean(residual_x**2 + residual_y**2)

def train_pinn(model, t_data, xy_data, n_epochs=10000):  # More epochs
    """Improved training process"""
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=100, factor=0.5)

    t_torch = torch.FloatTensor(t_data).reshape(-1, 1)
    xy_torch = torch.FloatTensor(xy_data)

    # Normalize data
    xy_scale = torch.max(torch.abs(xy_torch))
    xy_torch = xy_torch / xy_scale

    best_loss = float('inf')
    best_state = None
    losses = []

    for epoch in range(n_epochs):
        optimizer.zero_grad()

        xy_pred = model(t_torch)
        data_loss = torch.mean((xy_pred - xy_torch)**2)
        phys_loss = physics_loss(model, t_torch)

        # Dynamic weighting of losses
        physics_weight = 0.01 * (1 - np.exp(-epoch/1000))  # Gradually increase physics weight
        total_loss = data_loss + physics_weight * phys_loss

        total_loss.backward()
        optimizer.step()
        scheduler.step(total_loss)

        losses.append([total_loss.item(), data_loss.item(), phys_loss.item()])

        # Save best model
        if total_loss.item() < best_loss:
            best_loss = total_loss.item()
            best_state = model.state_dict().copy()

        if (epoch + 1) % 500 == 0:
            print(f'Epoch [{epoch+1}/{n_epochs}], '
                  f'Loss: {total_loss.item():.4f}, '
                  f'Data Loss: {data_loss.item():.4f}, '
                  f'Physics Loss: {phys_loss.item():.4f}')

    # Restore best model
    model.load_state_dict(best_state)
    return np.array(losses)

def main():
    # Generate data
    t_data, x_data, y_data = generate_orbital_data()
    xy_data = np.stack([x_data, y_data], axis=1)

    # Create and train model
    model = ImprovedPINN()
    losses = train_pinn(model, t_data, xy_data)

    # Plot losses
    plt.figure(figsize=(10, 4))
    plt.semilogy(losses[:, 0], label='Total Loss')
    plt.semilogy(losses[:, 1], label='Data Loss')
    plt.semilogy(losses[:, 2], label='Physics Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (log scale)')
    plt.legend()
    plt.grid(True)
    plt.show()

    # Evaluate predictions
    model.eval()
    with torch.no_grad():
        t_torch = torch.FloatTensor(t_data).reshape(-1, 1)
        xy_pred = model(t_torch).numpy()

    # Plot results
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.plot(xy_data[:, 0], xy_data[:, 1], 'b.', label='Data')
    plt.plot(xy_pred[:, 0], xy_pred[:, 1], 'r-', label='PINN')
    plt.plot(0, 0, 'y*', markersize=15)
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.legend()
    plt.axis('equal')
    plt.grid(True)

    plt.subplot(122)
    plt.plot(t_data, xy_data[:, 0], 'b.', label='Data X')
    plt.plot(t_data, xy_pred[:, 0], 'r-', label='PINN X')
    plt.xlabel('Time')
    plt.ylabel('Position')
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()

IndexError: index 1 is out of bounds for dimension 1 with size 1